In [ ]:
import os
import sys
sys.path.insert(0, "../..")
sys.path.insert(0, "../../../acquire")
import ipywidgets as widgets

from HUGS.Processing import search

from Acquire.Client import User, Drive, Service, PAR, Authorisation, StorageCreds

In [ ]:
# Autoreload modules before executing code, useful during development
%load_ext autoreload
%autoreload 2

## Start with uploading and processing of the data

In [ ]:
base_url = "https://hugs.acquire-aaai.com/t"
hugs_url = "https://hugs.acquire-aaai.com"
user = User(username="gareth", identity_url="%s/identity" % base_url)
response = user.request_login()

Login by visiting: https://login.acquire-aaai.com?id=a0-a6/49.b1.ce.e5
(please check that this page displays the message 'big cats climb quickly')


Check we're logged in

In [ ]:
user.wait_for_login()

True

In [ ]:
filename = "bsd.picarro.1minute.248m.dat"
dir_path = os.path.abspath("")
test_data = "../../test/data/proc_test_data/CRDS"
filepath = os.path.join(dir_path, test_data, filename)

hugs = Service(service_url="%s/hugs" % base_url)

creds = StorageCreds(user=user, service_url="%s/storage" % base_url)
drive = Drive(creds=creds, name="test_drive")
filemeta = drive.upload(filepath)
par = PAR(location=filemeta.location(), user=user)

par_secret = hugs.encrypt_data(par.secret())
auth = Authorisation(resource="process", user=user)

In [ ]:
def upload_file(authenticated_user, filepath):
    """ Upload a file to the object store
    
        Args:
            filepath (str): Path to file to upload
        Returns:
            None    
    """
    creds = StorageCreds(user=authenticated_user, service_url="%s/storage" % base_url)
    drive = Drive(creds=creds, name="test_drive")
    filemeta = drive.upload(filepath)
    par = PAR(location=filemeta.location(), user=user)

    par_secret = hugs.encrypt_data(par.secret())
    auth = Authorisation(resource="process", user=user)

In [ ]:
def get_CRDS_path(filename):
    dir_path = os.path.abspath("")
    test_data = "../../test/data/proc_test_data/CRDS"
    return os.path.join(dir_path, test_data, filename)

### Upload files for processing


Upload Bilsdale data

In [ ]:
bsd_path = get_CRDS_path("bsd.picarro.1minute.248m.dat")
upload_file(authenticated_user=user, filepath=bsd_path)


Clear the Datasources from CRDS and GC objects (currently Datasource UUIDs are just being randomly generated and we get multiple Datasources holding the same data)

In [ ]:
response = hugs.call_function(function="clear_datasources", args={})

In [ ]:
args = {"authorisation": auth.to_data(),
            "par": {"data": par.to_data()},
            "par_secret": {"data": par_secret},
            "data_type": "CRDS"}


response = hugs.call_function(function="process", args=args)

In [ ]:
print(response)

{'results': ['4a697871-fcf9-4d27-8d55-cce5ecb761c8', '7dcb8db4-d0a3-427b-a23e-7ea8d5352489', '852ea5e7-77b0-4cc0-a68d-4f306827f678']}


In [ ]:
from HUGS.Client import Process

filename = "bsd.picarro.1minute.248m.dat"
filepath = get_CRDS_path(filename)

processing = Process(service_url=base_url)
result = processing.process_files(user=user, files=filepath, data_type="CRDS")
print(result)

{'bsd.picarro.1minute.248m.dat': ['ccb3f0b8-9bee-4bb4-a766-213ee16a17b5', 'ff21298c-2fc5-4a33-81cf-a00f0cc9e236', 'a049f9e7-788c-4f4c-98d6-1f8f8d184165']}


Upload some Heathfield data

In [ ]:
filename = "hfd.picarro.1minute.100m_min.dat"
filepath = get_CRDS_path(filename)
result = processing.process_files(user=user, files=filepath, data_type="CRDS")
print(result)

{'hfd.picarro.1minute.100m_min.dat': ['4af8be1c-a048-45ab-8d08-46d3af126195', '6d4e64eb-852e-4839-a833-fcc22e542018', 'c979dbcf-5a81-4570-93cb-ee3c7f81053f']}


In [ ]:
from HUGS.Processing import search

search_results = None

search_terms = widgets.Text(
                    value="",
                    placeholder = "Search",
                    description = "Search terms:",
                    disabled = False
)

locations = widgets.Text(
                value="",
                placeholder = "BSD, HFD",
                description = "Locations:",
                disabled = False
)

data_type = widgets.Dropdown(
                options=["CRDS", "GC"], 
                value="CRDS",
                description="Data type",
                disabled=False
    )



layout=widgets.Layout(display = "flex", width = "50%")
search_button = widgets.Button(description="Search", button_style="success", \
                               layout=widgets.Layout(flex='1 1 0%', width='25%'))

start_picker = widgets.DatePicker(
                    description='Start date',
                    disabled=False
)

end_picker = widgets.DatePicker(
                    description='End date',
                    disabled=False
)


box = widgets.VBox(children=[search_terms, locations, start_picker, end_picker, data_type,
                                 search_button], layout=layout)



def call_search(x):
    """ Call the search function and pass it the values 
        in the text boxes
            
    """
    from datetime import datetime
    from Acquire.ObjectStore import datetime_to_string
    from HUGS.Client import Search
    start = datetime.combine(start_picker.value, datetime.min.time())
    end = datetime.combine(end_picker.value, datetime.min.time())
    
    split_search_terms = search_terms.value.replace(" ", "").split(",")
    split_locations = locations.value.replace(" ", "").split(",")
    
    global search_results
    search = Search(service_url=base_url)
    search_results = search.search(search_terms=split_search_terms, locations=split_locations, data_type=data_type.value, start_datetime=start, end_datetime=end)

search_button.on_click(call_search)




In [ ]:
box

In [ ]:
search_results.keys()

dict_keys(['bsd_co', 'hfd_co', 'bsd_co2', 'hfd_co2'])

Now take the result dictionary, parse the dictionary into a blocks that can be selected by the user

In [ ]:
def parse_results(results):
    """ Split the keys into a list of each key and the date that the data covers
        
        Args:
            results (dict): Dictionary of search results
        Returns:
            list (tuple): List of date, data key list pairs
    """
    date_keys = {}
    for key in results.keys():
        keys = sorted(results[key])
        start_key = keys[0]
        end_key = keys[-1]
        # Get the first and last dates from the keys in the search results
        start_date = start_key.split("/")[-1].split("_")[0]
        end_date = end_key.split("/")[-1].split("_")[-1]
        
        dates_covered = start_date + "_" + end_date
        
        date_keys[key] = {"dates": dates_covered, "keys": keys}
        
    return date_keys

Site, gas, dates and checkbox

In [ ]:
date_keys = parse_results(search_results)

checkboxes = []
date_labels = []
gas_labels = []
site_labels = []

for key in date_keys:
    checkbox = widgets.Checkbox(value=False, description=key, disabled=False)
    # Get the dates covered
    dates = date_keys[key]["dates"].replace("_", " ").replace("T", " ")
    date_label = widgets.Label(value=dates)
        
    split_key = key.split("_")
    site_name = split_key[0].upper()
    gas_name = split_key[1].upper()
    
    gas_label = widgets.Label(value=gas_name)
    site_label = widgets.Label(value=site_name)
    
    checkboxes.append(checkbox)
    date_labels.append(date_label)
    site_labels.append(site_label)
    gas_labels.append(gas_label)
    

In [ ]:
import ipywidgets as widgets

data = ["data1", "data2", "data3", "data4"]

checkboxes = [widgets.Checkbox(value=False, description=label) for label in data]

widgets.VBox(children=checkboxes)

In [ ]:
from ipywidgets import HBox, VBox

def changed():
    print("Wooo")
    
gas_box = VBox(children=gas_labels)
site_box = VBox(children=site_labels)
dates_box = VBox(children=date_labels)
checkbox_box = VBox(children=checkboxes)

hbox = HBox(children=[site_box, gas_box, dates_box, checkbox_box])

for v in gas_box.children:
    v.observe(changed)





In [ ]:
import ipywidgets as widgets

data = ["data1", "data2", "data3", "data4"]

selected_data = []

checkboxes = [widgets.Checkbox(value=False, description=label) for label in data]

widgets.VBox(children=checkboxes)

In [ ]:
widgets.Label(value="Yayah")

Label(value='Yayah')

In [ ]:
import ipywidgets as widgets
from IPython.display import display
x = 5
slider = widgets.IntSlider()
slider.value = x
def on_change(v):
    x = v['new'] 
slider.observe(on_change, names='value')
display(slider)

IntSlider(value=5)

In [ ]:
keys = search_results["results"]
data_args = {"return_type":"json", "keys": keys}
# Retrieve the data from the object store
data = hugs.call_function(function="retrieve", args=data_args)

KeyError: 'results'

In [ ]:
data

Now we can plot some of the data

In [ ]:
import pandas as pd
# co2_data = pd.read_json(data["results"]["bsd_co2"])
co_data = pd.read_json(data["results"]["bsd_ch4"])


In [ ]:
co_data.head(10)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

fig = plt.figure()
# ax = fig.add_subplot(211)
# ax.plot(co2_data.index.values, co2_data["co2 count"], color = "#4e79a7" )
ax = fig.add_subplot(111)
ax.plot(co_data.index.values, co_data["ch4 count"], color = "#59a14f")

plt.show()

In [ ]:
# Checkboxes for selecting which data we want to plot
# Select the gas, it goes and plots the 
# Simple line plot

In [ ]:
from ipyleaflet import (
    Map,
    Marker, MarkerCluster, TileLayer, ImageOverlay, GeoJSON,
    Polyline, Polygon, Rectangle, Circle, CircleMarker, Popup,
    SplitMapControl, WidgetControl,
    basemaps, basemap_to_tiles
)

from ipywidgets import HTML

In [ ]:
center = [54.2361, -4.548]
zoom = 5
m = Map(center=center, zoom=zoom)
mark_bsd = Marker(location=(54.942544, -1.369204))
mark_bsd.popup = HTML(value="Bilsdale")

mark_mhd = Marker(location=(53.20,-9.54))
mark_mhd.popup = HTML(value="Macehead")

mark_tac = Marker(location=(52.511, 1.155003))
mark_tac.popup = HTML(value="Tacolneston")

m += mark_bsd
m += mark_mhd
m += mark_tac

display(m)

In [ ]:
m